# Microprocessor Companies Valuation Model

## Objective
Develop a comprehensive valuation model for microprocessor companies using macroeconomic indicators and financial metrics.

In [5]:
from dotenv import load_dotenv
import snowflake.connector
from fredapi import Fred
import pandas as pd
import os

load_dotenv("./.env", override=True)
FRED_API_KEY = os.getenv("FRED_API_KEY")
START_DATE = '2009-01-01'
END_DATE = pd.to_datetime('today').strftime('%Y-%m-%d')
#END_DATE = (pd.to_datetime('today') - pd.Timedelta(days=1500)).strftime('%Y-%m-%d')
LIST_BRONZE_TABLES_MACROECONOMIC = ['Bronze_Consumption', 'Bronze_Investment', 'Bronze_Government_Spending', 'Bronze_Exports', 'Bronze_Imports', 'Bronze_Unemployed', 'Bronze_Labor_Force', 'Bronze_CPI', 'Bronze_Current_Account_Balance', 'Bronze_Public_Debt', 'Bronze_Interest_Rate', 'Bronze_FDI', 'Bronze_Labor_Force_Participation']
Table_Macro_Series_Mapping  = {
    'Bronze_Consumption': 'PCE',                     # Personal Consumption Expenditures (C)
    'Bronze_Investment': 'GPDIC1',                   # Gross Private Domestic Investment (I)
    'Bronze_Government_Spending': 'GCEC1',           # Government Consumption Expenditures (G)
    'Bronze_Exports': 'EXPGSC1',                     # Exports of Goods and Services (X)
    'Bronze_Imports': 'IMPGSC1',                     # Imports of Goods and Services (M)
    'Bronze_Unemployed': 'UNEMPLOY',                 # Number of Unemployed
    'Bronze_Labor_Force': 'CLF16OV',                 # Civilian Labor Force
    'Bronze_CPI': 'CPIAUCSL',                        # CPI for All Urban Consumers
    'Bronze_Current_Account_Balance': 'IEABCA',      # Current Account Balance
    'Bronze_Public_Debt': 'GFDEBTN',                 # Federal Debt: Total Public Debt
    'Bronze_Interest_Rate': 'DGS10',                 # Treasury Constant Maturity Rate
    'Bronze_FDI': 'ROWFDIQ027S',                     # Net FDI Flows
    'Bronze_Labor_Force_Participation': 'CIVPART'    # Civilian Labor Force Participation Rate
}

LIST_BRONZE_TABLES_MICROECONOMIC = ['Bronze_Micro_Household_Spending','Bronze_Micro_Small_Business_Loans', 'Bronze_Micro_Retail_Sales', 'Bronze_Micro_Personal_Income']
Table_Micro_Series_Mapping  = {
    'Bronze_Micro_Household_Spending': 'DPCERA3M086SBEA',       # Household Spending
    'Bronze_Micro_Small_Business_Loans': 'BUSLOANS',            # Small Business Loans
    'Bronze_Micro_Retail_Sales': 'RSAFS',                        # Retail Sales
    'Bronze_Micro_Personal_Income': 'PI'                        # Personal Income
}

def retrieve_data():
    manager = DataRetriever(
        api_key=FRED_API_KEY,
        start_date = START_DATE,
        end_date = END_DATE
    )
    df_micro = manager.retrieve_microdata_fred()
    df_macro = manager.retrieve_macrodata_fred()
    return df_macro, df_micro

class DataRetriever:
    def __init__(self, api_key, start_date, end_date):
        """Constructor for the DataRetriever class
        Args:
            api_key (str): FRED API key
            start_date (str): Start date for the data retrieval
            end_date (str): end date for the data retrieval
        """
        self.api_key = api_key
        self.fred = Fred(api_key=self.api_key)
        self.start_date = START_DATE
        self.end_date = END_DATE

    def retrieve_microdata_fred(self):
        fred = Fred(api_key=FRED_API_KEY)        
        for table_name in LIST_BRONZE_TABLES_MICROECONOMIC:
            # Get the corresponding FRED series based on the table name
            if table_name in Table_Micro_Series_Mapping:
                series_code = Table_Micro_Series_Mapping[table_name]
                # Fetch data from FRED using the appropriate series code and date range
                data = fred.get_series(series_code, observation_start=self.start_date, observation_end=self.end_date)
                Table_Micro_Series_Mapping[table_name] = data
            else:
                print(f"No matching FRED series found for table {table_name}")

        household_spending = Table_Micro_Series_Mapping['Bronze_Micro_Household_Spending'].resample('M').last()
        small_bussiness_loans = Table_Micro_Series_Mapping['Bronze_Micro_Small_Business_Loans'].resample('M').last()
        retails_sales = Table_Micro_Series_Mapping['Bronze_Micro_Retail_Sales'].resample('M').last()
        personal_income = Table_Micro_Series_Mapping['Bronze_Micro_Personal_Income'].resample('M').last()

        # Concatenate all series into a single DataFrame, aligning them by the common index (date)
        df = pd.concat([household_spending, small_bussiness_loans, retails_sales, personal_income], axis=1, join='inner')
        # Assign column names to the DataFrame
        df.columns = ['Bronze_Micro_Household_Spending', 'Bronze_Micro_Small_Business_Loans', 'Bronze_Micro_Retail_Sales', 'Bronze_Micro_Personal_Income']
        df.ffill(inplace=True)
        df['DateTime'] = df.index
        df['Date'] = pd.to_datetime(df['DateTime']).dt.date
        df = df.drop(columns=['DateTime'])
        df = df.rename(columns={'Date': 'DateTime'})
        # Fill any missing values with forward fill
        df.ffill(inplace=True)
        df = df.map(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)
        return df
    
    def retrieve_macrodata_fred(self):
        fred = Fred(api_key=FRED_API_KEY)
        for table_name in LIST_BRONZE_TABLES_MACROECONOMIC:
            # Get the corresponding FRED series based on the table name
            if table_name in Table_Macro_Series_Mapping:
                series_code = Table_Macro_Series_Mapping[table_name]
                # Fetch data from FRED using the appropriate series code and date range
                data = fred.get_series(series_code, observation_start=self.start_date, observation_end=self.end_date)
                Table_Macro_Series_Mapping[table_name] = data
            else:
                print(f"No matching FRED series found for table {table_name}")
        
        consumption = Table_Macro_Series_Mapping['Bronze_Consumption'].resample('M').last()
        investment = Table_Macro_Series_Mapping['Bronze_Investment'].resample('M').last()
        government_spending = Table_Macro_Series_Mapping['Bronze_Government_Spending'].resample('M').last()
        exports = Table_Macro_Series_Mapping['Bronze_Exports'].resample('M').last()
        imports = Table_Macro_Series_Mapping['Bronze_Imports'].resample('M').last()
        unemployed = Table_Macro_Series_Mapping['Bronze_Unemployed'].resample('M').last()
        labor_force = Table_Macro_Series_Mapping['Bronze_Labor_Force'].resample('M').last()
        cpi_series = Table_Macro_Series_Mapping['Bronze_CPI'].resample('M').last()
        current_account_balance = Table_Macro_Series_Mapping['Bronze_Current_Account_Balance'].resample('M').last()
        public_debt = Table_Macro_Series_Mapping['Bronze_Public_Debt'].resample('M').last()
        interest_rate = Table_Macro_Series_Mapping['Bronze_Interest_Rate'].resample('M').last()
        fdi = Table_Macro_Series_Mapping['Bronze_FDI'].resample('M').last()
        labor_force_participation = Table_Macro_Series_Mapping['Bronze_Labor_Force_Participation'].resample('M').last()
        
        # Concatenate all series into a single DataFrame, aligning them by the common index (date)
        df = pd.concat([consumption, investment, government_spending, exports, imports, unemployed, labor_force,
                        cpi_series, current_account_balance, public_debt, interest_rate, fdi, labor_force_participation], axis=1, 
                        join='inner')
        # Assign column names to the DataFrame
        df.columns = ['Consumption', 'Investment', 'Government_Spending', 'Exports', 'Imports', 
                    'Unemployed', 'Labor_Force', 'CPI', 'Current_Account_Balance', 'Public_Debt', 
                    'Interest_Rate', 'FDI', 'Labor_Force_Participation']
        
        # Calculate GDP using the formula GDP = C + I + G + (X - M)
        df['GDP'] = df['Consumption'] + df['Investment'] + df['Government_Spending'] + (df['Exports'] - df['Imports'])
        df.ffill(inplace=True)
        # Calculate the GDP Growth Rate
        df['GDP_Growth_Rate'] = df['GDP'].pct_change(fill_method=None) * 100
        # Calculate Inflation Rate using the CPI
        df['Inflation_Rate'] = df['CPI'].pct_change(fill_method=None) * 100
        # Calculate Unemployment Rate
        df['Unemployment_Rate'] = (df['Unemployed'] / df['Labor_Force']) * 100
        df['DateTime'] = df.index
        df['Date'] = pd.to_datetime(df['DateTime']).dt.date
        df = df.drop(columns=['DateTime'])
        df = df.rename(columns={'Date': 'DateTime'})
        # Fill any missing values with forward fill
        df.ffill(inplace=True)
        df = df.map(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)
        return df

df_macro_eco, df_micro_eco = retrieve_data()
#df_macro_eco = retrieve_data()

ModuleNotFoundError: No module named 'dotenv'